In [ ]:
######################################################################
#Description:
#This exercise shows the usefulness of using sql to add selective data
#(only data of interest) to a pandas dataframe. This saves a lot of memory.

#Here I am performing exploratory data analysis on a 'dvdrental' database
#which I downloaded during an udemy course on SQL.
#The exercises show here were not a part of the course, rather something
#that I learnt out of curiosity.

In [1]:
#pip install psycopg2


The following command must be run outside of the IPython shell:

    $ pip install psycopg2

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [66]:
import psycopg2 as pg2

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [92]:
conn = pg2.connect(host='localhost', database='dvdrental',user='postgres',password='indrika1')

In [93]:
cur = conn.cursor()

In [110]:
#Explore how many movies were sold in each category

cur.execute('SELECT category_id, count(film_id) FROM film_category group by category_id order by category_id')
data=np.array(cur.fetchall())
print(data)

#plot the data

#plt.plot(data[:,0], data[:,1], 'r.', ms=6)
#plt.show()

[[ 1 64]
 [ 2 66]
 [ 3 60]
 [ 4 57]
 [ 5 58]
 [ 6 68]
 [ 7 62]
 [ 8 69]
 [ 9 73]
 [10 61]
 [11 56]
 [12 51]
 [13 63]
 [14 61]
 [15 74]
 [16 57]]


In [146]:
##############################################################################
### Find which customers paid the most for dvdrental in the last three months
#############################################################################
data = pd.read_sql("""
                   select 
                          customer.customer_id, first_name, last_name, address_id,
                          sum(payment.amount) as total_amount, 
                          extract(month from payment.payment_date) as month
                    from 
                          customer
                    inner join 
                          payment on customer.customer_id = payment.customer_id 
                    group by 
                          month, customer.customer_id
                    order by 
                          total_amount desc;
                    """, conn)

print("min month in data (year is 2007 for full data) : ", min(data["month"]))
print("max month in data (year is 2007 for full data) : ", max(data["month"]))

print(data.head(10))

min month in data (year is 2007 for full data) :  2.0
max month in data (year is 2007 for full data) :  5.0
   customer_id first_name last_name  address_id  total_amount  month
0          148    Eleanor      Hunt         152        100.78    4.0
1          522     Arnold    Havens         528         97.81    4.0
2          470     Gordon    Allard         475         96.83    4.0
3          137     Rhonda   Kennedy         141         96.81    4.0
4          144      Clara      Shaw         148         93.82    4.0
5          459      Tommy   Collazo         464         89.82    4.0
6          526       Karl      Seal         532         89.80    4.0
7          257     Marsha   Douglas         262         88.82    4.0
8          295      Daisy     Bates         300         88.81    4.0
9          148    Eleanor      Hunt         152         87.82    3.0


In [148]:
#Print the data of the customers who paid the max amount in last 3 months

for i in range(3,6):
    df_month = data[(data["month"]==i)]
    df_max = df_month[df_month["total_amount"]==max(df_month["total_amount"])]
    
    print("Information of highest paying customer in month", i, "of year 2007:")
    print(df_max.head(1))
    print("\n")
    
    

Information of highest paying customer in month 3 of year 2007:
   customer_id first_name last_name  address_id  total_amount  month
9          148    Eleanor      Hunt         152         87.82    3.0


Information of highest paying customer in month 4 of year 2007:
   customer_id first_name last_name  address_id  total_amount  month
0          148    Eleanor      Hunt         152        100.78    4.0


Information of highest paying customer in month 5 of year 2007:
      customer_id first_name last_name  address_id  total_amount  month
1562           60    Mildred    Bailey          64          9.98    5.0


